# Dynamic and interactive (matplotlib) plots

In today's class we will illustrate a few methods of interactive and dynamic plots, mostly with the **matplotlib** library.

## Topic 1 - a simple progress indicator

Say you have a lengthy computation and want to illustrate the progress made, so you don't get too impatient.  For example, the magnetic field flow lines from last class.  We will use the [**tqdm**](https://pypi.python.org/pypi/tqdm) library for this.

In [ ]:
import sys
expaths = ["/usr/lib/python3/dist-packages", "/usr/local/lib/python3.5/dist-packages"]
for xp in expaths:
    if (xp not in sys.path):
        sys.path.append(xp)

import tqdm
import time as ti

## gui based progress monitor
for i in tqdm.tqdm_notebook(range(12)):
    ti.sleep(0.5)

In [ ]:
## text-based progress monitor
## this can be used in a plain python session (without i-python / jupyter)
for i in tqdm.tqdm(range(6)):
    ti.sleep(1)    

## Topic 2 - very basic interactive plots


In [ ]:
from IPython.html.widgets import interact as ita

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

I=np.linspace(0,2*np.pi,300) ## the interval [0,2π]

def pltsin(k):
    plt.plot(I, np.sin(k*I))
    plt.title("Plot of sin(kx) on interval [0,2$\pi$], k="+str(k))

## k is a triple consisting of:
## (min value for k, max value, step size)
ita(pltsin, k=(1,10,0.1))

## TRY: adding a second argument

## Topic 3 -- a single pendulum

You have likely seen this differential equation in a mathematics or physics course,

$$\theta'' + a \theta' + b \sin \theta = 0.$$

it is the differential equation of a pendulum with friction, provided $a \geq 0$ and $b > 0$.  $a$ is *roughly* the friction while $b$ is *roughly* the mass. 

<img src="pendulum.png" alt="Riemann sum example" width=400 height=400>

We convert this $2^{nd}$ order ODE into the two-variable first order ODE,

$$\pmatrix{\theta \cr \theta'}' = \pmatrix{\theta' \cr -a \theta' - b \sin \theta}$$

which we will simulate (numerically) via the **scipy.integrate** **odeint** command.

In [ ]:
import numpy as np
import math as ma
from scipy.integrate import odeint

## I need this to get the unicode theta to render. 
import matplotlib as mpl
mpl.rc('font', family='DejaVu Sans')

## Let's create a routine to plot the solution to the ODE, for a given initial condition
## th0  is the initial theta value
## thp0 is the initial theta' value
## L    is the time-domain interval length
## n    is the number of steps
## a    is the friction constant a>=0
## b    is the mass constant b>0.
##
## returns a list of pairs (theta, theta') of length n
def ODEsol(th0, thp0, L, n, a, b):
    t=np.linspace(0.0, L, n)
    sol = odeint(lambda th, t, a, b: [th[1], -a*th[1]-b*np.sin(th[0])], [th0, thp0], t, args=(a, b))
    return sol

def plotSol(th0, thp0, L, n, a, b):
    sol = ODEsol(th0, thp0, L, n, a, b)
    ## let's add some code to indicate the relevant theta = 2*pi*k lines, for k an integer
    THvals = [x[0] for x in sol]
    Mi = ma.floor(min(THvals) / (2*np.pi) )
    Ma = ma.ceil( max(THvals) / (2*np.pi) )
    ytk = [2*np.pi*i for i in range(Mi, Ma+1)]
    for y in ytk:
        plt.plot( [0, L], [y, y], 'r--')
    ## and the appropriate y-ticks
    plt.yticks(ytk, [str(2*k)+'π' for k in range(Mi, Ma+1)])
    ## and the plot.
    plt.plot(np.linspace(0, L, n), THvals, 'b-')
    plt.xlabel('time')
    plt.ylabel('θ')
    
## make an interactive plot
ita(plotSol, th0=(-np.pi, np.pi, 0.1), thp0=(-10, 10, 0.1), L=(1, 100, 0.2),\
                 n=(10, 5000, 1), a=(0, 2, 0.01), b=(1, 10, 1))

While we're on the topic, let's do an elementary check on how *good* our simulation is.  At *first glance* it does appear to be a pretty good simulation -- the zero friction case reproduces something that looks quite similar to a periodic function. 

One way to check if a system appears roughly correct would be to check if the total energy of the system behaves as expected.  Since this system has friction, one would suspect that there is always a *decrease* in the total energy, due only to the friction. 

A more *math-y* approach would be to look for a Lyapunov function for the system.  Either way leads one to suspect that a linear combination of the two functions:

$$ (\theta')^2 \hskip 1cm \text{ and } \hskip 1cm \cos \theta $$

will product a function $E(t)$ which has negative derivative.  A little calculus and we discover the function

$$ E(t) = (\theta')^2 - 2b \cos \theta $$

does the job.  Differentiating gives

$$ E'(t) = 2 \theta' \theta'' + 2b (\sin \theta) \theta' = 2 \theta' (-a\theta' -b\sin \theta) + 2b (\sin \theta) \theta' = -2a (\theta')^2$$

Let's check the accuracy of this for our simulation. We have our simulations's value for $\theta'(t)$ in the variables **x[1]** for **x** in **THvals**.   We can also compute the (numerical) derivative of $E$, given our sequence of points **THvals**.  We plot both and compare. 

The code below makes the same plot, but gives the **MDPL** which is the maxmum of the deviations of the power loss, theoretical vs. what appears in the simulation.  The other number will be the **ADPL** which is the average deviation in theoretical vs. experimental power loss. 

In [ ]:
def plotSol(th0, thp0, L, n, a, b):
    sol = ODEsol(th0, thp0, L, n, a, b)
    ## let's add some code to indicate the relevant theta = 2*pi*k lines, for k an integer
    THvals = [x[0] for x in sol]
    THpvals = [x[1] for x in sol]
    Mi = ma.floor(min(THvals) / np.pi )
    Ma = ma.ceil( max(THvals) / np.pi )
    ytk = [np.pi*i for i in range(Mi, Ma+1)]
    for y in ytk:
        plt.plot( [0, L], [y, y], 'r--' if (int(y/np.pi) % 2 == 0) else 'y--')

    ## and the appropriate y-ticks
    plt.yticks(ytk, [str(k)+'π' for k in range(Mi, Ma+1)])
    
    ## let's compute the expected power loss, and plot
    elossT = [2*a*x[1]**2 for x in sol] # theoretical energy loss
    elossS = [((THpvals[i-1]**2 - 2*b*np.cos(THvals[i-1]))-\
               (THpvals[i+1]**2 - 2*b*np.cos(THvals[i+1])))/(2*L/n) for i in range(1,n-1)]## loss in simulation
    #plt.plot(np.linspace(0,L,n), elossT, 'r-') 
    #xarr = [i*L/n for i in range(1, n-1)]
    #plt.plot( xarr, elossS, 'y-')    
    maxEL = max([abs(elossT[i]-elossS[i-1]) for i in range(1, n-1)])
    avgEL = sum([abs(elossT[i]-elossS[i-1]) for i in range(1, n-1)]) / (n-2)
    ## and the plot.
    plt.plot(np.linspace(0, L, n), THvals, 'b-')
    plt.xlabel('time')
    plt.ylabel('θ')
    plt.title('MaxDPL %.3f, AvgDPL %.3f' % (maxEL, avgEL))
    
## make an interactive plot
ita(plotSol, th0=(-np.pi, np.pi, 0.1), thp0=(-10, 10, 0.1), L=(1, 100, 0.2),\
                 n=(10, 5000, 1), a=(0, 2, 0.01), b=(1, 10, 1))


## Topic 4 -- Let's visualize the pendulum. . . as a pendulum. 

Plotting $\theta$ vs. $t$ is nice, but let's get something that looks like the *real* thing.  We will create a time-evolving display that looks something like an actual pendulum. 

Do do this, we will use the **matplotlib.animation** library.  Roughly speaking, one sets up a normal **matplotlib** figure as you usually would, and we call the **animation.FuncAnimation** call, giving it an *update* function which updates the figure, and refreshes the screen, this makes **animation.FuncAnimation** responsible for performing the animation, and the *update* function the guiding instructions for how the animation evolves.

We will start with a minimalist pendulum example, and elaborate on it with a second example.


In [ ]:
plt.close()
import matplotlib.animation as animation
%matplotlib nbagg

sol = ODEsol(0.0, 4.2, 80, 10000, 0.1, 1)

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-1, 1), ylim=(-1, 1) )
ax.grid()

line1, = ax.plot([],[],'b-', lw=4)

en_text1 = ax.text(0.02,0.92, '', transform=ax.transAxes, fontsize=16)

def init():
    line1.set_data([],[])
    en_text1.set_text('')
    return line1, en_text1

def animate( TH ):
    line1.set_data( [0.0, np.sin(TH[0])], [0.0 , -np.cos(TH[0])] )
    en_text1.set_text('$\\theta$ = %.1f $\\pi$' % (TH[0]/np.pi) )
    
    return line1, en_text1

ani = animation.FuncAnimation(fig, animate, frames = sol, init_func=init,\
                              interval=1, blit=True)

plt.show()

**Let's decorate** our animation with some additional details, such as the current variable values, and some details to illustrate these, visually. 

In [ ]:
plt.close()
sol = ODEsol(0.0, 4.2, 80, 10000, 0.1, 1)

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-1.1, 1.1), ylim=(-1.1, 1.1) )
ax.grid()

line1, = ax.plot([],[],'b-', lw=4)
line2, = ax.plot([],[],'r-', lw=2)

en_text1 = ax.text(0.02, 0.92, '', transform=ax.transAxes, fontsize=16)
en_text2 = ax.text(0.02, 0.82, '', transform=ax.transAxes, fontsize=16)

def init():
    line1.set_data([],[])
    line2.set_data([],[])
    en_text1.set_text('')
    en_text2.set_text('')
    return line1, line2, en_text1, en_text2

def animate( TH ):
    line1.set_data( [0.0, np.sin(TH[0])], [0.0 , -np.cos(TH[0])] )
    line2.set_data( [np.sin(TH[0]), np.sin(TH[0]) + np.cos(TH[0])*TH[1]/3],\
                    [-np.cos(TH[0]),-np.cos(TH[0])+ np.sin(TH[0])*TH[1]/3] )
    en_text1.set_text('$\\theta$ = %.1f $\\pi$' % (TH[0]/np.pi) )
    en_text2.set_text('$\\theta\'$ = %.1f ' % TH[1] )
    
    return line1, line2, en_text1, en_text2

ani = animation.FuncAnimation(fig, animate, frames = sol, init_func=init,\
                              interval=1, blit=True)

plt.show()

## Topic 5 -- A double pendulum!

Simulating a double pendulum brings in several new complications.  First of all, transcribing the ODE is simply difficult! 

<img src="doublepen.gif" width="300" height="300" class="alignleft"/> </div>

It's difficult because the ODE is relatively long.  Rather than look the ODE up in a book (which is a bit of an adventure) we will use a tool from Physics (with help from Sympy) to derive the ODE. 

In [ ]:
import sympy as sp
from sympy.utilities.autowrap import ufuncify

m1,m2,k1,k2,r1,r2,g,t = sp.symbols("m_1 m_2 k_1 k_2 r_1 r_2 g t")
t1 = sp.Function("θ_1")(t)
t2 = sp.Function("θ_2")(t)
w1 = sp.Derivative(t1, t)
w2 = sp.Derivative(t2, t)
ti = sp.Function("θ_i")(t)
wi = sp.Function("w_i")(t)

R = sp.Function("R")
L = sp.Function("L")

T, V = sp.symbols("T V")

Teq = sp.Eq(T, (1/2)*m1*r1**2*(w1**2) + m2*r1*r2*w1*w2*sp.cos(t1-t2)+(1/2)*m2*r1**2*(w1**2)+\
      (1/2)*m2*r2**2*(w2**2))
Veq = sp.Eq(V, -m1*g*r1*sp.cos(t1)-m2*g*(r1*sp.cos(t1)+r2*sp.cos(t2)))
Req = sp.Eq(R, (1/2)*k1*w1**2 + (1/2)*k2*(w2-w1)**2)
# L = T - V
Leq = sp.Eq(L, Teq.args[1] - Veq.args[1])
print(" * * * The set-up * * * \n")
print("Kinetic Energy")
sp.pprint(Teq)
print("\nPotential Energy")
sp.pprint(Veq)
print("\nRayleigh dissipation term")
sp.pprint(Req)
print("\nLagrangian L = T - V")
sp.pprint(Leq)
print("\nLagrange's Equations of Motion")
sp.pprint( sp.Eq(sp.Derivative(L, wi, t) - sp.Derivative(L,ti) +\
           sp.Derivative(R, wi), 0))
print("for i=1,2")

In [ ]:
## Computing with the values of L and R. 
LEM1 = sp.Eq(sp.Derivative(Leq.args[1], w1, t).doit() - \
             sp.Derivative(Leq.args[1], t1).doit() + \
             sp.Derivative(Req.args[1], w1).doit(), 0).simplify()
LEM2 = sp.Eq(sp.Derivative(Leq.args[1], w2, t).doit() - \
             sp.Derivative(Leq.args[1], t2).doit() + \
             sp.Derivative(Req.args[1], w2).doit(), 0).simplify()
print("First eqn of motion:")
sp.pprint(LEM1)
print("\n\nSecond eqn of motion:")
sp.pprint(LEM2)
#look good.

In [ ]:
print("Lagrange's Equations of Motion \n")
print("Solving for θ''_1, θ''_2")
SOL = sp.solve((LEM1, LEM2), sp.Derivative(t1, t, 2), sp.Derivative(t2, t, 2))
sp.pprint(SOL)
#looks good.

In [ ]:
T1pp = SOL[sp.Derivative(t1,t,2)].simplify()
T2pp = SOL[sp.Derivative(t2,t,2)].simplify()

print("θ''_1 = ")
sp.pprint(T1pp)

print("\nθ''_2 = ")
sp.pprint(T2pp)

In [ ]:
## As a test, let's see if dE/dt = -2R. 
## this is how Lagrangians work for "conservative" frictional
## forces such as ours.
test = sp.Derivative(Teq.args[1]+Veq.args[1], t).doit().doit()
test = test.simplify()
test = test.xreplace({sp.Derivative(t1,t,2): T1pp, sp.Derivative(t2,t,2): T2pp})
test = test.simplify()
print("This should be zero. . .")
sp.pprint( (test+2*Req.args[1]).simplify() )
## good!

In [ ]:
## before we cast T1pp and T2pp let's simplify them a bit.  We are replacing
## the unicode with ASCII since in the next pane we want to use ufuncify, 
## which chokes on unicode.

w1, w2, t1, t2 = sp.symbols("w_1 w_2 θ_1 θ_2")
xT1pp = T1pp
xT1pp = xT1pp.xreplace({sp.Derivative(sp.Function("θ_1")(t),t): w1, sp.Derivative(sp.Function("θ_2")(t),t): w2} )
xT1pp = xT1pp.xreplace({sp.Function("θ_1")(t): t1, sp.Function("θ_2")(t) : t2})
#sp.pprint(xT1pp)

xT2pp = T2pp
xT2pp = xT2pp.xreplace({sp.Derivative(sp.Function("θ_1")(t),t): w1, sp.Derivative(sp.Function("θ_2")(t),t): w2} )
xT2pp = xT2pp.xreplace({sp.Function("θ_1")(t): t1, sp.Function("θ_2")(t) : t2})
#sp.pprint(xT2pp)

t1,t2 = sp.symbols("t1 t2")
xT1pp = xT1pp.xreplace({sp.Symbol("θ_1") : t1, sp.Symbol("θ_2") : t2})
xT2pp = xT2pp.xreplace({sp.Symbol("θ_1") : t1, sp.Symbol("θ_2") : t2})
print("θ''_1 = . . . in ASCII ")
sp.pprint(xT1pp)
print("\n\nθ''_2 = . . . in ASCII ")
sp.pprint(xT2pp)


In [ ]:
## be careful -- ufuncify does not like unicode characters!
t1pp = ufuncify([r1,r2, m1,m2, k1,k2, t1,t2, w1,w2, g], xT1pp )
t2pp = ufuncify([r1,r2, m1,m2, k1,k2, t1,t2, w1,w2, g], xT2pp )

In [ ]:
## this is our ODE.  It takes as input theta1, theta2, theta1', and theta2' and
## returns theta1'' and theta2''. It also needs r1,r2, m1,m2, k1,k2. 
## It incodes the above. 
## TH = [t1,t2,t1p, t2p]
def theODE2(TH, t, r1,r2, m1,m2, k1,k2, g):
    return [TH[2], TH[3],\
            float(t1pp(r1,r2, m1,m2, k1,k2, TH[0],TH[1], TH[2],TH[3], g)),\
            float(t2pp(r1,r2, m1,m2, k1,k2, TH[0],TH[1], TH[2],TH[3], g))]


In [ ]:
plt.close()
import copy

## Code to generate an approximate solution to the double pendulum ODE.
##
## returns a list of pairs (theta0, theta0', theta1, theta1') of length n
def ODEsol2(th0, thp0, th1, thp1, L, n, m1, m2, r1, r2, k1, k2, g):
    t=np.linspace(0.0, L, n)
    sol = odeint( theODE2, [th0, th1, thp0, thp1], t, args=(r1,r2, m1,m2, k1,k2, g) )
    return sol

def plotSol(th0, thp0, th1, thp1, L, m1, m2, r1, r2, k1, k2, g, n):
    sol = ODEsol2(th0, thp0, th1, thp1, L, n, m1, m2, r1, r2, k1, k2, g)
    ## let's add some code to indicate the relevant theta = 2*pi*k lines, for k an integer
    TH0vals = [x[0] for x in sol]
    TH1vals = [x[1] for x in sol]
    Yvals = copy.copy(TH0vals)
    Yvals.extend(TH1vals)
    
    Mi = ma.floor(min(Yvals) / np.pi )
    Ma = ma.ceil( max(Yvals) / np.pi )
    ytk = [np.pi*i for i in range(Mi, Ma+1)]
    for y in ytk:
        plt.plot( [0, L], [y, y], 'r--' if (int(y/np.pi) % 2==0) else 'y--' )
    ## and the appropriate y-ticks
    plt.yticks(ytk, [str(k)+'π' for k in range(Mi, Ma+1)])
    ## and the plot.
    plt.plot(np.linspace(0, L, n), TH0vals, 'b-')
    plt.plot(np.linspace(0, L, n), TH1vals, 'g-')
    plt.xlabel('time')
    plt.ylabel('θ')
    
##      th0, thp0,th1,thp1,     L, m1,   m2,  r1,  r2,  k1,  k2,  g,   n
plotSol(0.0, 3.0, 0.0,-9.0, 120.0, 1.0, 1.0, 1.0, 1.0, 0.01, 0.01, 6, 10000)

## Topic 6 -- animated double pendulum

Let's visualize the solution as a real double-pendulum, evolving in time. 

We will simply cut and paste our code from Topic 4. 

In [ ]:
## animated plot of double pendulum. 
th0=0; thp0=3.3; th1=0; thp1=4; L=200; n=10000; m1=1; m2=1; r1=1; r2=1; k1=0.1; k2=0.1; g=5

plt.close()
sol = ODEsol2(th0, thp0, th1, thp1, L, n, m1, m2, r1, r2, k1, k2, g)
    
fig = plt.figure()
prad = 1.1*(r1+r2)
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-prad, prad), ylim=(-prad, prad) )
ax.grid()

line1, = ax.plot([],[],'b-', lw=3)
en_text1 = ax.text(0.02, 0.92, '', transform=ax.transAxes, fontsize=16)
en_text2 = ax.text(0.02, 0.82, '', transform=ax.transAxes, fontsize=16)

def init():
    line1.set_data([],[])
    en_text1.set_text('')
    en_text2.set_text('')
    return line1, en_text1, en_text2

def animate( TH ):
    line1.set_data( [0.0, r1*np.sin(TH[0]), r1*np.sin(TH[0])+r2*np.sin(TH[1])],\
                    [0.0 , -r1*np.cos(TH[0]), -r1*np.cos(TH[0])-r2*np.cos(TH[1])] )
    en_text1.set_text('$\\theta_0$ = %.1f $\\pi$' % (TH[0]/np.pi) )
    en_text2.set_text('$\\theta_1$ = %.1f $\\pi$' % (TH[1]/np.pi) )
    return line1, en_text1, en_text2

ani = animation.FuncAnimation(fig, animate, frames = sol, init_func=init,\
                              interval=1, blit=True)

plt.show()


In [ ]:
## Let's similarly cast V, T and R. 
V = Veq.args[1]
R = Req.args[1]
T = Teq.args[1]
Vp = V
Vp = Vp.xreplace({sp.Function("θ_1")(t): t1, sp.Function("θ_2")(t) : t2})
Rp = R
Rp = Rp.xreplace({sp.Derivative(sp.Function("θ_1")(t),t): w1, sp.Derivative(sp.Function("θ_2")(t),t): w2} )
Rp = Rp.xreplace({sp.Function("θ_1")(t): t1, sp.Function("θ_2")(t) : t2})
Tp = T
Tp = Tp.xreplace({sp.Derivative(sp.Function("θ_1")(t),t): w1, sp.Derivative(sp.Function("θ_2")(t),t): w2} )
Tp = Tp.xreplace({sp.Function("θ_1")(t): t1, sp.Function("θ_2")(t) : t2})
## let's further normalize Vp to be zero at the stable fixed point
Vp = Vp + g*m1*r1 + g*m2*(r1+r2)
#sp.pprint(Vp)
Vc = ufuncify([m1,m2, r1,r2, t1,t2, g], Vp)
Rc = ufuncify([k1,k2, w1,w2],Rp)
Tc = ufuncify([m1,m2, r1,r2, t1,t2, w1,w2], Tp)

In [ ]:
## kinetic + potential energy
def compE(m1,m2,r1,r2,t1,t2,t1p,t2p,g):
    return float(Vc(m1,m2,r1,r2,t1,t2,g)) + float(Tc(m1,m2,r1,r2,t1,t2,t1p,t2p))
        
## theoretical rate of change of energy should be -2*Rayleigh function. 
def dEdt(k1,k2,t1p,t2p):
    return -2*float(Rc(k1,k2,t1p,t2p))
